In [25]:

from user_token import USER_TOKEN
import requests
import json

def do_elastic_query(query):
    body = json.dumps(query, indent=4)
    headers = {"Content-Type": "application/json", "Authorization": "Bearer " + USER_TOKEN}
    url = "http://localhost:9200/rs1/_search"
    res = requests.get(data=body, headers=headers, url=url)
    return res

In [26]:
query = {
    "query": {
        "bool": {
            "must":
                [
                    {
                        "match": {
                            "RestaurantName": "hot"}
                    },
                    {
                        "range": {
                            "AggregateRating": {
                                "gte": 4,
                                "lte": 5
                            }
                        }
                    }
                ],
            "must_not":
                {
                    "match": {
                        "City": "Abu Dhabi"
                    }
                }
        }
    }
}

res = do_elastic_query(query)
res.json()

{'took': 84,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 1, 'relation': 'eq'},
  'max_score': 6.4178452,
  'hits': [{'_index': 'rs1',
    '_type': '_doc',
    '_id': '_pirZYABikYBi-qbiyGa',
    '_score': 6.4178452,
    '_source': {'RestaurantID': '15008',
     'RestaurantName': 'Hot Breads',
     'City': 'Ludhiana',
     'AverageCostForTwo': 800.0,
     'AggregateRating': 4.2,
     'RatingText': 'Very Good',
     'Votes': 261.0,
     'Date': '2016-04-12 15:58:03',
     'Location': [75.82172044, 30.89323422]}}]}}

In [27]:
query = {
  "query": {
    "bool": {
      "must": {
        "match_all": {}
      },
      "filter": {
        "geo_distance": {
          "distance": "5km",
          "Location": {
            "lat": 28.6139,
            "lon": 77.209
          }
        }
      }
    }
  },
  "sort": [
    {
      "AggregateRating": {
        "order": "desc"
      }
    }
  ]
}

res = do_elastic_query(query)
res.json()

{'took': 202,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 992, 'relation': 'eq'},
  'max_score': None,
  'hits': [{'_index': 'rs1',
    '_type': '_doc',
    '_id': 'LJirZYABikYBi-qbiySb',
    '_score': None,
    '_source': {'RestaurantID': '310143',
     'RestaurantName': 'Naturals Ice Cream',
     'City': 'New Delhi',
     'AverageCostForTwo': 150.0,
     'AggregateRating': 4.9,
     'RatingText': 'Excellent',
     'Votes': 2620.0,
     'Date': '2017-08-30 11:33:49',
     'Location': [77.2221482, 28.6343484]},
    'sort': [4.9]},
   {'_index': 'rs1',
    '_type': '_doc',
    '_id': 'LpirZYABikYBi-qbiymd',
    '_score': None,
    '_source': {'RestaurantID': '18345728',
     'RestaurantName': 'Masala Library',
     'City': 'New Delhi',
     'AverageCostForTwo': 5000.0,
     'AggregateRating': 4.9,
     'RatingText': 'Excellent',
     'Votes': 408.0,
     'Date': '2018-08-25 14:07:01',
     'Location': [77.218645

In [28]:
query = {
  "query": {
    "regexp": {
      "RestaurantName": {
        "value": ".*ind.*",
        "flags": "ALL",
        "case_insensitive": True,
        "max_determinized_states": 10000
      }
    }
  },
  "highlight": {
    "fields": {
      "RestaurantName": {  }
    }
  },
  "_source": {
    "includes": [
      "RestaurantName",
      "City",
      "AggregateRating"
    ]
  }
}

res = do_elastic_query(query)
res.json()

{'took': 804,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 178, 'relation': 'eq'},
  'max_score': 1.0,
  'hits': [{'_index': 'rs1',
    '_type': '_doc',
    '_id': '_5irZYABikYBi-qbixiV',
    '_score': 1.0,
    '_source': {'City': 'Augusta',
     'RestaurantName': 'The Taj of India',
     'AggregateRating': 4.0},
    'highlight': {'RestaurantName': ['The Taj of <em>India</em>']}},
   {'_index': 'rs1',
    '_type': '_doc',
    '_id': 'IJirZYABikYBi-qbixmV',
    '_score': 1.0,
    '_source': {'City': 'Cedar Rapids/Iowa City',
     'RestaurantName': 'Taste of India',
     'AggregateRating': 4.2},
    'highlight': {'RestaurantName': ['Taste of <em>India</em>']}},
   {'_index': 'rs1',
    '_type': '_doc',
    '_id': 'JJirZYABikYBi-qbixmV',
    '_score': 1.0,
    '_source': {'City': 'Cedar Rapids/Iowa City',
     'RestaurantName': 'Exotic India',
     'AggregateRating': 4.1},
    'highlight': {'RestaurantName': ['Exot

In [29]:
query = {
  "query": {
    "regexp": {
      "RestaurantName": {
        "value": ".*delicious.*",
        "flags": "ALL",
        "case_insensitive": True,
        "max_determinized_states": 10000
      }
    }
  },
  "aggs": {
    "rating_range": {
      "range": {
        "field": "AggregateRating",
        "ranges": [
          {
            "to": 3
          },
          {
            "from": 3,
            "to": 4
          },
          {
            "to": 5
          }
        ]
      },
      "aggs": {
        "docs": {
          "top_hits": {
            "size": 10
          }
        }
      }
    }
  }
}

res = do_elastic_query(query)
res.json()

{'took': 398,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 9, 'relation': 'eq'},
  'max_score': 1.0,
  'hits': [{'_index': 'rs1',
    '_type': '_doc',
    '_id': 'ZZirZYABikYBi-qbiyCZ',
    '_score': 1.0,
    '_source': {'RestaurantID': '18462602',
     'RestaurantName': 'Delicious Food Corner',
     'City': 'Gurgaon',
     'AverageCostForTwo': 400.0,
     'AggregateRating': 0.0,
     'RatingText': 'Not rated',
     'Votes': 0.0,
     'Date': '2016-10-04 16:57:29',
     'Location': [77.0947125, 28.4810852]}},
   {'_index': 'rs1',
    '_type': '_doc',
    '_id': '85irZYABikYBi-qbiyac',
    '_score': 1.0,
    '_source': {'RestaurantID': '311690',
     'RestaurantName': 'MR.D - Deliciousness Delivered',
     'City': 'New Delhi',
     'AverageCostForTwo': 700.0,
     'AggregateRating': 3.3,
     'RatingText': 'Average',
     'Votes': 148.0,
     'Date': '2016-05-23 01:34:27',
     'Location': [77.2457421, 28.5281318

In [30]:
query = {
  "size": 0,
  "query": {
    "range": {
      "Votes": {
        "gte": 1000
      }
    }
  },
  "aggs": {
    "ByCity": {
      "terms": {
        "field": "City.keyword",
        "min_doc_count": 10
      },
      "aggs": {
        "AverageCostForTwo": {
          "avg": {
            "field": "AverageCostForTwo"
          }
        },
        "CityBucketSort": {
          "bucket_sort": {
            "sort": [
              {
                "AverageCostForTwo": {
                  "order": "desc"
                }
              }
            ],
            "size": 7
          }
        }
      }
    }
  }
}

res = do_elastic_query(query)
res.json()

{'took': 232,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 291, 'relation': 'eq'},
  'max_score': None,
  'hits': []},
 'aggregations': {'ByCity': {'doc_count_error_upper_bound': 0,
   'sum_other_doc_count': 5,
   'buckets': [{'key': 'Gurgaon',
     'doc_count': 23,
     'AverageCostForTwo': {'value': 1852.1739130434783}},
    {'key': 'Bangalore',
     'doc_count': 11,
     'AverageCostForTwo': {'value': 1390.909090909091}},
    {'key': 'New Delhi',
     'doc_count': 119,
     'AverageCostForTwo': {'value': 1311.3445378151262}},
    {'key': 'Kolkata',
     'doc_count': 16,
     'AverageCostForTwo': {'value': 1253.125}},
    {'key': 'Chennai',
     'doc_count': 14,
     'AverageCostForTwo': {'value': 1164.2857142857142}},
    {'key': 'Mumbai',
     'doc_count': 11,
     'AverageCostForTwo': {'value': 1118.1818181818182}},
    {'key': 'Tampa Bay',
     'doc_count': 13,
     'AverageCostForTwo': {'value': 33.076923

In [31]:
query = {
  "size": 0,
  "aggs": {
    "ByReviewType": {
      "terms": {
        "field": "RatingText.keyword"
      },
      "aggs": {
        "ReviewTypeBucketSort": {
          "bucket_sort": {
            "sort": [
              {
                "_count": {
                  "order": "desc"
                }
              }
            ]
          }
        }
      }
    }
  }
}

res = do_elastic_query(query)
res.json()

{'took': 136,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 9499, 'relation': 'eq'},
  'max_score': None,
  'hits': []},
 'aggregations': {'ByReviewType': {'doc_count_error_upper_bound': 0,
   'sum_other_doc_count': 0,
   'buckets': [{'key': 'Average', 'doc_count': 3719},
    {'key': 'Not rated', 'doc_count': 2143},
    {'key': 'Good', 'doc_count': 2094},
    {'key': 'Very Good', 'doc_count': 1068},
    {'key': 'Excellent', 'doc_count': 291},
    {'key': 'Poor', 'doc_count': 184}]}}}